In [1]:
!pip install configparser
!pip install pandas
!pip install statistics
#!pip install transformers
#!pip install datasets
import pandas as pd
import numpy as np
import requests
import os
import re
import csv
import glob
import sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
display(pd.read_excel("/content/drive/MyDrive/fyp/fyp2/manual labels/person 1 (liven)/labellingdata1.xlsx"))

,Unnamed: 0,Unnamed: 0.1,date,tweet,is_retweet,real_score
0,1790341.0,1796849.0,2021-11-18 19:52:08,🔥🔥🔥 ANOTHER MASSIVE WIN! 🔥🔥🔥\n\nLUNA/USD Take-...,False,5.0
1,1328213.0,1333600.0,2021-10-23 09:37:54,&gt;There is no inflation\n&gt;There is some i...,False,3.0
2,2995648.0,3004265.0,2022-04-19 04:12:41,China’s largest broadcaster’s NFT series becom...,False,3.0
3,106747.0,106789.0,2021-05-29 22:54:12,❗Bee Network-Cloud Mining on Your Phone ❗☁️⛏️💰...,False,4.0
4,2446463.0,2453852.0,2022-03-18 21:04:16,"Have you heard of $PIT, one of the largest com...",False,3.0
...,...,...,...,...,...,...
95,1067250.0,1071559.0,2021-08-16 07:41:18,"dogecoin is,according to and . I would not b...",False,4.0
96,1315769.0,1321112.0,2021-10-23 17:55:43,They are going to crash the economy to replace...,False,3.0
97,2356505.0,2363820.0,2022-03-10 15:20:24,You can do Cloud Mining using this site ...,False,6.0
98,2030020.0,2036873.0,2022-01-13 18:58:56,What doesn't Bitcoin fix?,False,2.0


In [3]:
#path = "/content/drive/MyDrive/fyp/fyp2/manual labels/person 1 (liven)"
path = "/content/drive/MyDrive/fyp/fyp2/manual labels"
path_turk = "/content/drive/MyDrive/fyp/fyp2/Amazon turks results"
out_name = "manual_label_Gold"
out_name_dataset = 'dataset_new_with_self_label_final'
out_folder = '/content/drive/MyDrive/fyp/fyp2/data with sentient scores'
out_folder_dataset = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset'


turn_nonsense_tweet_to_neutral = True

In [4]:


def ConvertScore2Label(x,y):
 # print(y)
  if isinstance(y,str) and  y == 'true':
    return "spam"
  else:
    if x > 0:
      return 'Positive'
    elif x < 0:
      return 'Negative'
    elif x == 0:
      return 'Neutral'
    else:
      print('error')
      return np.nan

In [5]:


df_list = []


def concat_df_in_folder(path):
  all_files = glob.glob( (path + "/*.xlsx") )
  li = []
  for filename in all_files:
      df = pd.read_excel(filename, index_col=None, header=0)
      li.append(df)
  frame = pd.concat(li, axis=0, ignore_index=True)
  return frame


all_folders = glob.glob( (path + "/*") , recursive = True)
#print(all_folders)

for folder in all_folders:
  df = concat_df_in_folder(folder)
  df['spam'] = df.real_score.apply(lambda x: 'true' if x == 6 else 'false')
  #display(df)
  df.replace([6,1,2,3,4,5],[0,-1.0,-0.5,0,0.5,1.0],inplace = True)
  df.set_index('tweet')
  df_list.append(df)

#for df in df_list:
#  display(df)
  


df_concat = pd.concat(df_list)
by_row_index = df_concat.groupby(df_concat.index).agg({'date': 'first', 'tweet': 'first', 'is_retweet': 'first', 'real_score': 'mean','spam': pd.Series.mode}).reset_index()

df_manual = by_row_index

#df_manual['label'] = df_manual.real_score.spam.apply(lambda x ,y : ConvertScore2Label(x,y))
df_manual['label'] = df_manual.apply(lambda x: ConvertScore2Label(x.real_score, x.spam), axis=1)

display(df_manual)

#df_means.to_csv(out_folder + '/' + out_name + '.csv')

,index,date,tweet,is_retweet,real_score,spam,label
0,0,2021-11-18 19:52:08,🔥🔥🔥 ANOTHER MASSIVE WIN! 🔥🔥🔥\n\nLUNA/USD Take-...,False,1.00,false,Positive
1,1,2021-10-23 09:37:54,&gt;There is no inflation\n&gt;There is some i...,False,0.00,false,Neutral
2,2,2022-04-19 04:12:41,China’s largest broadcaster’s NFT series becom...,False,0.00,false,Neutral
3,3,2021-05-29 22:54:12,❗Bee Network-Cloud Mining on Your Phone ❗☁️⛏️💰...,False,0.00,false,Neutral
4,4,2022-03-18 21:04:16,"Have you heard of $PIT, one of the largest com...",False,0.25,false,Positive
...,...,...,...,...,...,...,...
295,295,2022-01-12 09:21:10,NFT news.\n\nKakao Entertainment to expand its...,False,0.00,true,spam
296,296,2021-04-24 12:24:23,Crypto mining as it should be! \n\nZionodes pr...,False,0.00,true,spam
297,297,2021-07-21 04:19:31,"The state now has 1,350 active locally-acquire...",False,0.00,"[false, true]",Neutral
298,298,2021-10-19 17:25:28,What’s amazing it’s not just for the rich and ...,False,0.25,false,Positive


In [6]:
#d = {'text': df_means['tweet'], 'label': df_means['real_score']}
#df = pd.DataFrame(data=d)

In [7]:
#df = df_.copy()

def Convert2Label_turk(x):
  if x == "Very Negative" or x == "Negative":
    return "Negative"
  elif x == "Very Positive" or x == "Postive":
    return "Positive"
  elif x == "Neutral":
    return "Neutral"
  elif x == "N/A or spam or unrelated":
    return "Neutral"
  else:
    print("error ", x)
    return


def multimode(list1):
    import statistics
    list_table = statistics._counts(list1)
    len_table = len(list_table)
    new_list = []
    for i in range(len_table):
        new_list.append(list_table[i][0])
    return new_list 


def process_df_turk(df_,incomplete = False):
  from collections import Counter
  
  def count_(li):
    return len(li)

  d = {'text': df_['Input.text'], 'label': df_['Answer.sentiment analysis for bitcoin tweets.label']}
  df = pd.DataFrame(data=d)
  #print(df.shape[0])
  df['label'] = df.label.apply(lambda x: Convert2Label_turk(x))
  
  #df['spam_score'] = df.label_score.apply(lambda x: 'true' if x == "N/A or spam or unrelated" else 'false')
  #df = df.groupby('text').mean().reset_index()
  #df['label_score'] = df.label_score.apply(lambda x: Score2Label(x))
  #df = df.groupby('text').count().reset_index()
  #df = df.groupby(['text'])['label'].agg(pd.Series.cumsum).to_frame()
  df = df.groupby('text')['label'].apply(list).to_frame()
  #print(df.shape[0])
  df['label_count'] = df['label'].apply(count_)
  #df = df[df.label_count > 1]
  df['label_mode'] = df['label'].apply(multimode)
  df['decided_label'] = df['label_mode'].apply(lambda x: x[0] if len(x) == 1  else  "remove" )
  #print(df.shape[0])
  df = df[df['decided_label'] != "remove"]
  df = df.reset_index()
  df = df[['text','decided_label','label_count']]
  df = df.rename(columns={"decided_label": "label"})
  #df = df[df['label_count'] > 1]

  
  if incomplete == True:
    df_train = df[df['label_count']==2]
    df_train = df_train[['text','label']]

    df_valid = df[df['label_count'] > 2]
    df_valid = df_valid[['text','label']]
    return df_train,df_valid

  elif incomplete == False:
    return df[['text','label']]
  else:
    print("error")
    return 



def concat_amazon_data(path,incomplete = False):
  all_folders = glob.glob( (path + "/*") , recursive = True)
  #print(all_folders)
  #display(process_df_turk(df))

  li = []
  li_valid = []

  #print(all_folders)
  for filename in all_folders:
      df = pd.read_csv(filename)
      if incomplete == False:
        df = process_df_turk(df,incomplete)
        li.append(df)
      else:
        df_train,df_valid = process_df_turk(df,incomplete)
        li.append(df_train)
        li_valid.append(df_valid)
  if incomplete:
    train = pd.concat(li)
    valid = pd.concat(li_valid)
    return train,valid
  else: 
    df_turk = pd.concat(li)
    return df_turk  

df_turk =concat_amazon_data(path_turk)

display(df_turk)
#print(df_turk)

,text,label
0,"\n$34,316.01\n\n -0.28%\n +1.35%\n +7.83%\n +3...",Positive
1,\n1. NUPL at .434 in the yellow\n2. Thermocap ...,Negative
2,\nUSD 29185.02 | EUR 27252.49 | JPY 3719732.88,Neutral
3,\n\n\n\ncrytpo dao minecraft cryptocurrency c...,Neutral
4,"\n30 PM . Mar 09,2022)\n\nBTC $41921.88\nETH ...",Neutral
...,...,...
86,bitcoins prices will not be transitory,Neutral
87,fideliscrypto crypto investment fidelis btc c...,Neutral
88,for more!,Neutral
89,"givingaway 10,000 jasmy away to 5 lucky follow...",Neutral


In [8]:
#join turk and personal labels

def decide_ambigious_spam_columns(x,label):
  if isinstance(x,str):
    return label
  elif isinstance(x,np.ndarray):
    return 'spam'

#df_turk = df_turk_.copy()

df_personal = df_manual[['tweet','label','spam']]

df_personal['label'] = df_personal.apply(lambda x: decide_ambigious_spam_columns(x.spam,x.label),axis = 1 )
df_personal.drop(columns = 'spam',inplace = True)

df_personal.rename(columns={"tweet": "text"},inplace = True)
#df_pe

#display(df_personal)
li_full = [df_personal,df_turk]
print(df_personal.shape[0])
print(df_turk.shape[0])

df_full = pd.concat(li_full,ignore_index = True)
display(df_manual)
display(df_turk)
display(df_full)

#df_turk.to_csv('/content/drive/MyDrive/fyp/fyp2/amazon turks concat/full_amazon.csv' )
#df_full.to_csv('/content/drive/MyDrive/fyp/fyp2/labelled data concat' + '/full_labelled_data.csv')

300
91


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,index,date,tweet,is_retweet,real_score,spam,label
0,0,2021-11-18 19:52:08,🔥🔥🔥 ANOTHER MASSIVE WIN! 🔥🔥🔥\n\nLUNA/USD Take-...,False,1.00,false,Positive
1,1,2021-10-23 09:37:54,&gt;There is no inflation\n&gt;There is some i...,False,0.00,false,Neutral
2,2,2022-04-19 04:12:41,China’s largest broadcaster’s NFT series becom...,False,0.00,false,Neutral
3,3,2021-05-29 22:54:12,❗Bee Network-Cloud Mining on Your Phone ❗☁️⛏️💰...,False,0.00,false,Neutral
4,4,2022-03-18 21:04:16,"Have you heard of $PIT, one of the largest com...",False,0.25,false,Positive
...,...,...,...,...,...,...,...
295,295,2022-01-12 09:21:10,NFT news.\n\nKakao Entertainment to expand its...,False,0.00,true,spam
296,296,2021-04-24 12:24:23,Crypto mining as it should be! \n\nZionodes pr...,False,0.00,true,spam
297,297,2021-07-21 04:19:31,"The state now has 1,350 active locally-acquire...",False,0.00,"[false, true]",Neutral
298,298,2021-10-19 17:25:28,What’s amazing it’s not just for the rich and ...,False,0.25,false,Positive


,text,label
0,"\n$34,316.01\n\n -0.28%\n +1.35%\n +7.83%\n +3...",Positive
1,\n1. NUPL at .434 in the yellow\n2. Thermocap ...,Negative
2,\nUSD 29185.02 | EUR 27252.49 | JPY 3719732.88,Neutral
3,\n\n\n\ncrytpo dao minecraft cryptocurrency c...,Neutral
4,"\n30 PM . Mar 09,2022)\n\nBTC $41921.88\nETH ...",Neutral
...,...,...
86,bitcoins prices will not be transitory,Neutral
87,fideliscrypto crypto investment fidelis btc c...,Neutral
88,for more!,Neutral
89,"givingaway 10,000 jasmy away to 5 lucky follow...",Neutral


,text,label
0,🔥🔥🔥 ANOTHER MASSIVE WIN! 🔥🔥🔥\n\nLUNA/USD Take-...,Positive
1,&gt;There is no inflation\n&gt;There is some i...,Neutral
2,China’s largest broadcaster’s NFT series becom...,Neutral
3,❗Bee Network-Cloud Mining on Your Phone ❗☁️⛏️💰...,Neutral
4,"Have you heard of $PIT, one of the largest com...",Positive
...,...,...
386,bitcoins prices will not be transitory,Neutral
387,fideliscrypto crypto investment fidelis btc c...,Neutral
388,for more!,Neutral
389,"givingaway 10,000 jasmy away to 5 lucky follow...",Neutral


In [9]:
df_improved_amazon_train = concat_amazon_data('/content/drive/MyDrive/fyp/fyp2/amazon turk improved labels/train result/raw')
display(df_improved_amazon_train)
#df_improved_amazon_train.to_csv('/content/drive/MyDrive/fyp/fyp2/amazon turk improved labels/train result/concat/improved_amazon_concat_train.csv')



df_improved_amazon_valid = concat_amazon_data('/content/drive/MyDrive/fyp/fyp2/amazon turk improved labels/valid-test result /raw')
display(df_improved_amazon_valid)
#df_improved_amazon_valid.to_csv('/content/drive/MyDrive/fyp/fyp2/amazon turk improved labels/valid-test result /concat/improved_amazon_concat_valid')

,text,label
0,\n\n Funds Increase Bitcoin $BTC Exposure\n R...,Neutral
1,"\n\n $39,309.32\n $1.186\n $389.10\n $0.8356\n...",Neutral
2,"\n\n BTC identified from CryptoMichNL's Tweet,...",Neutral
3,\n\nBTC ETH BNB SOL ADA \n\n $BTC $43140.45 (1...,Neutral
4,\n\nBTCMarkets\n $88748\n $841\n $5385\n $265\...,Neutral
...,...,...
138,i love you \n\n Shibaverse \n\n_io \n \nBTC \n...,Neutral
139,it is really awesome that you have a lot of B...,Positive
140,stacking here Bitcoin,Neutral
141,the project looks promising and all the criter...,Neutral


,text,label
0,"\n55 PM . Apr 18,2022)\n\nBTC $40711.27\nETH ...",Neutral
1,NFTGiveway Coming Soon \n Sloths ...,Neutral
2,WOWOWOW SMH!\n\nBTC\nBitcoin ADA XRPArmy cryp...,Neutral
3,late night left 4 dead !\n\nLeft4Dead HIVE Tw...,Neutral
4,$CHZ\nAn unusual limit order (sell) added to t...,Neutral
...,...,...
274,this is the best smart and very strong project...,Neutral
275,we can earn money by buying their token on les...,Neutral
276,who is in . . . .Follow me and I will follow y...,Neutral
277,why all BTC holders are so arrogant?,Negative


In [10]:
df_incomplete_train,df_incomplete_valid = concat_amazon_data('/content/drive/MyDrive/fyp/fyp2/amazon turk improved labels/incomplete result/raw',incomplete = True)
display(df_incomplete_train)
display(df_incomplete_valid)

,text,label
1,) cryptocurrencies are the future and no count...,Positive
4,AUCTIONUSDT Bull Alert!\n\n5X Volume\n 15...,Positive
5,As start than we will have diner ? BTC,Neutral
8,BTC DOGE Why Is Dogecoin Tumbling Today?,Negative
10,BTCUSDT \n41332.85000 \n14 AM\n 67.37797Fiftee...,Positive
13,"ClickBait - guess it works screaming FIRE, FI...",Positive
17,I buy Bitcoin every time the Fear &amp; Greed ...,Positive
24,TODAY IS THE DAY ..\n\nBITCOIN TO THE MOON\n\n...,Positive
26,"The $BTC price is at $42,309.85 right now.\n C...",Negative
29,Want you your own PERSONAL BNB Mining Rig? Get...,Neutral


,text,label
0,"$1,881,736,086,334\n $40,500.26\n 40.92%\n $3...",Neutral
2,"1 Bitcoin ( BTC )\n 57,004.95$ \n\n1 Bitcoin C...",Neutral
3,1180) Current Bitcoin Price is $40920 BTC Cryp...,Neutral
6,AscendEX is excited to launch the ASDP Auction...,Positive
7,BTC \n\nDUMP,Negative
...,...,...
81,the strongest bullish signal flashed for Bitco...,Positive
82,the strongest bullish signal flashed for Bitco...,Positive
83,whre is the $dot whales\ndont miss these coins...,Positive
84,you are always talking about Bitcoin but MarkM...,Neutral


In [11]:

df_manual_self = pd.read_json("/content/drive/MyDrive/fyp/fyp2/amazon turk improved labels/manual_label_self/export-2022-08-05T05_00_39.265Z.json")
display(df_manual_self)



,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Updated At,Seconds to Label,External ID,Agreement,Benchmark Agreement,Benchmark ID,Dataset Name,Reviews,View Label,Has Open Issues,Skipped
0,cl6f4b6ob1sub08zy7jvx35b1,cl6f4ae5g0knq075a72dsg90f,afternoon goodafternoon goodvibes affiliatemar...,"{'objects': [], 'classifications': [{'featureI...",mahmudur.rahman17@imperial.ac.uk,text bitcoin tweet,2022-08-05T04:58:20.000Z,2022-08-05T04:58:20.512Z,11.527,NaN,-1,-1,NaN,v2_amazon_data_train_m17.csv,[],https://editor.labelbox.com?project=cl6f497f51...,0,False
1,cl6f4h96216ge07xog75qej1q,cl6f4ae4f0jqi075agv4e7vpj,"(BTC) \nWazirx\n 1BTC = 4,633,471 INR\nCoin...","{'objects': [], 'classifications': [{'featureI...",mahmudur.rahman17@imperial.ac.uk,text bitcoin tweet,2022-08-04T14:17:54.000Z,2022-08-04T14:17:54.478Z,36.751,NaN,-1,-1,NaN,v2_amazon_data_train_m17.csv,[],https://editor.labelbox.com?project=cl6f497f51...,0,False
2,cl6f4h9vgb75407ys5blx6p2u,cl6f4ae4g0jqm075adigwbv18,This is a excellent project. I am confident pr...,"{'objects': [], 'classifications': [{'featureI...",mahmudur.rahman17@imperial.ac.uk,text bitcoin tweet,2022-08-04T14:18:44.000Z,2022-08-04T14:18:44.592Z,52.569,NaN,-1,-1,NaN,v2_amazon_data_train_m17.csv,[],https://editor.labelbox.com?project=cl6f497f51...,0,False
3,cl6f4hntx263x07z75h7p1mkc,cl6f4ae4g0jqq075acqa0c7c1,"Biggoals and dreams. God willing, I help other...","{'objects': [], 'classifications': [{'featureI...",mahmudur.rahman17@imperial.ac.uk,text bitcoin tweet,2022-08-04T14:18:56.000Z,2022-08-04T14:18:56.418Z,11.341,NaN,-1,-1,NaN,v2_amazon_data_train_m17.csv,[],https://editor.labelbox.com?project=cl6f497f51...,0,False
4,cl6f4iqf3afln07ze3gah5zvk,cl6f4ae4g0jqu075a0qp55ex9,"\n55 PM . Mar 20,2022)\n\nBTC $41343.96\nETH ...","{'objects': [], 'classifications': [{'featureI...",mahmudur.rahman17@imperial.ac.uk,text bitcoin tweet,2022-08-04T14:19:01.000Z,2022-08-04T14:19:02.905Z,44.692,NaN,-1,-1,NaN,v2_amazon_data_train_m17.csv,[],https://editor.labelbox.com?project=cl6f497f51...,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,cl6fv1ipo85ir07zi0jx9d3iw,cl6f4ae5g0kn6075a1v6d2kq0,"BTC Bitcoin Bitcoin Key To Twitter's Future, S...","{'objects': [], 'classifications': [{'featureI...",mahmudur.rahman17@imperial.ac.uk,text bitcoin tweet,2022-08-05T04:57:21.000Z,2022-08-05T04:57:21.767Z,521.300,NaN,-1,-1,NaN,v2_amazon_data_train_m17.csv,[],https://editor.labelbox.com?project=cl6f497f51...,0,False
296,cl6fv1ns18pk6082g05kncp77,cl6f4ae5g0kna075aclbh0pi8,How Antifragility Increases Bitcoin&amp;039;s ...,"{'objects': [], 'classifications': [{'featureI...",mahmudur.rahman17@imperial.ac.uk,text bitcoin tweet,2022-08-05T04:57:33.000Z,2022-08-05T04:57:33.991Z,11.450,NaN,-1,-1,NaN,v2_amazon_data_train_m17.csv,[],https://editor.labelbox.com?project=cl6f497f51...,0,False
297,cl6fzwncma7cp070s77wp6hvr,cl6f4ae5g0kne075a09bu5a6t,Annnnd ... remember you can also get paid cryp...,"{'objects': [], 'classifications': [{'featureI...",mahmudur.rahman17@imperial.ac.uk,text bitcoin tweet,2022-08-05T04:57:42.000Z,2022-08-05T04:57:42.841Z,6.869,NaN,-1,-1,NaN,v2_amazon_data_train_m17.csv,[],https://editor.labelbox.com?project=cl6f497f51...,0,False
298,cl6fzwwtr0pnx0714h2djeh3g,cl6f4ae5g0kni075aakcmap3k,$GALA Another 800%+ trade and we making anothe...,"{'objects': [], 'classifications': [{'featureI...",mahmudur.rahman17@imperial.ac.uk,text bitcoin tweet,2022-08-05T04:57:57.000Z,2022-08-05T04:57:57.109Z,13.572,NaN,-1,-1,NaN,v2_amazon_data_train_m17.csv,[],https://editor.labelbox.com?project=cl6f497f51...,0,False


In [12]:

def extract_label(Label):
  return Label.get("classifications")[0].get("answer").get("title")

df_manual_self['label'] = df_manual_self['Label'].apply(extract_label)

df_manual_self = df_manual_self[['Labeled Data','label']].rename(columns={"Labeled Data": "text"}) 

display(df_manual_self)

,text,label
0,afternoon goodafternoon goodvibes affiliatemar...,Neutral
1,"(BTC) \nWazirx\n 1BTC = 4,633,471 INR\nCoin...",Neutral
2,This is a excellent project. I am confident pr...,Positive
3,"Biggoals and dreams. God willing, I help other...",Neutral
4,"\n55 PM . Mar 20,2022)\n\nBTC $41343.96\nETH ...",Neutral
...,...,...
295,"BTC Bitcoin Bitcoin Key To Twitter's Future, S...",Positive
296,How Antifragility Increases Bitcoin&amp;039;s ...,Positive
297,Annnnd ... remember you can also get paid cryp...,Neutral
298,$GALA Another 800%+ trade and we making anothe...,Neutral


In [13]:
df_whole_original = pd.read_csv("/content/drive/MyDrive/fyp/Bitcoin_tweets_cleaned_2.csv")[['tweet']].dropna().drop_duplicates(subset= 'tweet',keep="first")

def modified_2_original(df_convert,df_whole_orginal=df_whole_original):
 
  df_whole_orginal['text'] = df_whole_orginal['tweet']
  new = df_whole_orginal[['text']]
  new = new.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
  #print(type(new))
  #display(new)
  df_whole_orginal['modded_text'] = new['text']


  df_test = df_convert.copy().rename(columns={"text": "text2"})#.drop_duplicates(subset=['text2'],keep = "first")
  #df_whole_orginal = df_whole_orginal.drop_duplicates(subset=['modded_text'],keep = "first")

  #display(df_test.shape[0])
  #display(df_whole_orginal.shape[0])

  merged = pd.merge(df_test, df_whole_orginal,left_on='text2', right_on='modded_text' , how='inner').drop_duplicates(subset= 'text',keep="first")
  if 'quality' in merged.columns:
    return merged[['text','label','quality']].drop_duplicates(subset= 'text',keep="first")
  else:
    return merged[['text','label']].drop_duplicates(subset= 'text',keep="first")


  

def frac_df(df_,frac):
 df = df_.sample(frac=1) 
 size = df.shape[0]
 #print(frac)
 f1 = df[0:int(size*frac[0])]
 f2 = df[int(size*frac[0]):int(size*frac[1])+int(size*frac[0])]
 f3 = df[int(size*frac[1])+int(size*frac[0]):]
 #print(size)
 #print(int(f1.shape[0])+int(f2.shape[0])+int(f3.shape[0]))
 return [f1,f2,f3],size

def create_dataset(df_manual,df_old_turk,df_train,df_valid,df_incomplete_train,df_incomplete_valid,df_manual_self,frac=[0.1,0.6,0.3]):
  df_train["source"] = "new_turk_low"
  df_manual["source"] = "manual"
  df_old_turk["source"] = "old_turk"
  df_valid["source"] = "new_turk_high"
  df_incomplete_train["source"] = "incomplete_train"
  df_incomplete_valid["source"] = "incomplete_valid"
  df_manual_self["source"] = "self_manual"


  df_train["quality"] = "low"
  df_manual["quality"] = "high"
  df_old_turk["quality"] = "high"
  df_valid["quality"] = "high"
  df_incomplete_train["quality"] = "low"
  df_incomplete_valid["quality"] = "high"
  df_manual_self["quality"] = "high"




  def split_half(df):
    size = df.shape[0]
    size_half = int(size /2)
    df1 = df[0:size_half]
    df2 = df[size_half:]
    return df1,df2

  #fraction all high quality data
  if frac[0] + frac[1] + frac[2] != 1:
    frac=[0.2,0.5,0.3]
    print("error, frac not equal to one so defaulting to: ", frac)
    
 

  frac_manual,s1 = frac_df(df_manual,frac)
  frac_old_turk,s2 = frac_df(df_old_turk,frac)
  frac_valid,s3 = frac_df(df_valid,frac)
  frac_manual_self,s4 = frac_df(df_manual_self,frac)

  print("total size of the split data:", s1+s2+s3+s4)

  

  train = df_train
  train = train.append(df_incomplete_train)
  train = train.append(frac_manual[0])
  train = train.append(frac_old_turk[0])
  train = train.append(frac_valid[0])
  train = train.append(frac_manual_self[0])

  valid_incomp1 , valid_incomp2 = split_half(df_incomplete_valid)
  
  valid = frac_manual[1]
  valid = valid.append(frac_old_turk[1])
  valid = valid.append(frac_valid[1])
  valid = valid.append(valid_incomp1)
  valid = valid.append(frac_manual_self[1])

  test = frac_manual[2]
  test = test.append(frac_old_turk[2])
  test = test.append(frac_valid[2])
  test = test.append(valid_incomp2)
  test = test.append(frac_manual_self[2])


  #train = train.rename_axis('tweet').reset_index()

  return train.sample(frac=1),valid.sample(frac=1),test.sample(frac=1)



frac = [0.5,0.3,0.2]
train,valid,test =create_dataset(df_manual[['tweet','label']].rename(columns={"tweet": "text"}) , modified_2_original(df_turk) ,modified_2_original(df_improved_amazon_train), modified_2_original(df_improved_amazon_valid), modified_2_original(df_incomplete_train), modified_2_original(df_incomplete_valid),modified_2_original(df_manual_self),frac=frac)


print(df_manual.shape[0]+df_turk.shape[0]+df_improved_amazon_train.shape[0]+df_improved_amazon_valid.shape[0])


def rep(string):
  #print(string)
  #print(type(string))
  try:
    string_ = clean(string)
    string_ = string_.replace('\n'," ")
    string_ = string_.replace('\t'," ")
    
    return string_
  except:
    return string

def clean(text):
  text = re.sub(r'@[A-Za-z0-9:]+', '',text) #removes @ mentions
  text = re.sub(r'#','',text) #remove #
  text = re.sub(r'RT[\s]+','',text) # remove RT
  text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', text) #removes links
  text = re.sub(r'\s*$','',text) #remove leading and trailing whitespace
  text = re.sub(r'^\s*','',text) #remove leading and leading whitespace
  text = re.sub(r'.+:', '',text) #remove username:
  return text



train['text'] = train.text.apply(rep)
valid['text'] = valid.text.apply(rep)
test['text'] = test.text.apply(rep)
print(train.shape[0]+valid.shape[0]+test.shape[0])
display(train)
display(valid)
display(test)
save = False
if save:
  print("saving datasets")
  out_datasets_ = out_folder_dataset + "/" + out_name_dataset + "_" + str(frac[0]*10) + ":"+ str(frac[1]*10) + ":"+ str(frac[1]*10) 
  if not os.path.exists(out_datasets_):
    os.makedirs(out_datasets_)

  print(out_datasets_)
  train.to_csv(out_datasets_+ "/train_dataset.csv")
  valid.to_csv(out_datasets_+ "/valid_dataset.csv")
  test.to_csv(out_datasets_+ "/test_dataset.csv")
  print("saving successful")




/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


total size of the split data: 1802
3084
5005


,text,label,source,quality
1027,_Matt African variant can cost bitcoin to cras...,Positive,new_turk_low,low
1437,"100k Bitcoin is inevitable, after 100k it's un...",Negative,new_turk_low,low
699,Full thanks to as they have analysed this Bit...,Neutral,new_turk_high,high
1861,If Bitcoin closes with the weekly candle abov...,Positive,new_turk_low,low
497,12 $BTC sold into the bid @ 34111 👈 Bitcoin...,Neutral,new_turk_high,high
...,...,...,...,...
610,🌹List your token on FegEx. Just DM . ListOnFEG...,Neutral,new_turk_low,low
842,MATIC__Polygon CRYPTO MARKET CRYPTO BUY_|...,Neutral,new_turk_high,high
303,"__crypto _tw 🔥 SanjiInu 🔥 💥 KYC, CHARITY, NF...",Neutral,self_manual,high
1544,Full thanks to as they have analysed this Bit...,Positive,new_turk_low,low


,text,label,source,quality
280,_vlz bluesparrow will take over the world T...,Neutral,incomplete_valid,high
349,Listing on coinbase couple of months ago .. /...,Neutral,incomplete_valid,high
276,"Putting utility and longevity aside, I need to...",Positive,manual,high
239,I am very thankful to for knowing me this dan...,Negative,new_turk_high,high
395,New intraday forecasts for EURUSD GBPUSD(Free)...,Neutral,incomplete_valid,high
...,...,...,...,...
461,Huuuuge News!!! New Utility Income Breakdown i...,Neutral,new_turk_high,high
202,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high
74,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high
264,This zesty faucet from @_bitcoiner is making m...,Neutral,incomplete_valid,high


,text,label,source,quality
825,LEOPARD TOKEN🚀 🎯🎯 ✅Liquidity is locked ✅Certi...,Neutral,incomplete_valid,high
323,$FTM is DIPPING! (z.score=-2.15) @ $2.47 11.4%...,Negative,self_manual,high
606,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high
548,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high
509,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high
...,...,...,...,...
530,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high
660,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high
780,💰💰Full thanks to as they have analysed this B...,Neutral,new_turk_high,high
340,A heartly thanks to for giving me right tips...,Positive,new_turk_high,high


saving datasets
/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_final_5.0:3.0:3.0
saving successful


In [14]:
display(df_turk)
display(modified_2_original(df_turk))

,text,label
0,"\n$34,316.01\n\n -0.28%\n +1.35%\n +7.83%\n +3...",Positive
1,\n1. NUPL at .434 in the yellow\n2. Thermocap ...,Negative
2,\nUSD 29185.02 | EUR 27252.49 | JPY 3719732.88,Neutral
3,\n\n\n\ncrytpo dao minecraft cryptocurrency c...,Neutral
4,"\n30 PM . Mar 09,2022)\n\nBTC $41921.88\nETH ...",Neutral
...,...,...
86,bitcoins prices will not be transitory,Neutral
87,fideliscrypto crypto investment fidelis btc c...,Neutral
88,for more!,Neutral
89,"givingaway 10,000 jasmy away to 5 lucky follow...",Neutral


,text,label
0,"\n$34,316.01\n\n -0.28%\n +1.35%\n +7.83%\n +3...",Positive
1,\n1. NUPL at .434 in the yellow\n2. Thermocap ...,Negative
2,\nUSD 29185.02 | EUR 27252.49 | JPY 3719732.88,Neutral
3,\n\n\n\ncrytpo dao minecraft cryptocurrency c...,Neutral
4,"\n30 PM . Mar 09,2022)\n\nBTC $41921.88\nETH ...",Neutral
...,...,...
83,bitcoin is love\n\nThose who hate it...\n\n......,Positive
84,bitcoin’s prices will not be transitory,Neutral
85,fideliscrypto crypto investment fidelis btc c...,Neutral
86,"givingaway 10,000 jasmy away to 5 lucky follow...",Neutral


In [15]:
#display(df_turk)
#.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
display(pd.read_csv("/content/drive/MyDrive/fyp/fyp2/amazon turk improved labels/modified(unlabeled)-train/v2_amazon_data_train_m0.csv"))
display(pd.read_csv("/content/drive/MyDrive/fyp/fyp2/amazon turk improved labels/original(unlabeled)-train/v2_amazon_data_train_o0.csv"))


,index,text
0,1617175,_73 Thank you for the opportunity to participa...
1,722687,"5 Top Crypto to Buy this Week [BTC, ETH, ADA, ..."
2,1142504,Huobi The World's 2nd Largest Cryptocurrency ...
3,1767599,99%+ of traders don't care about Ferraris and ...
4,164807,I love this project so much! I hope the projec...
...,...,...
295,1692263,"Episode 11 is live and Bibles, Bullets, Bitcoi..."
296,3298451,Bitcoin tumbles more than 50% below its all-ti...
297,2286522,FaviTrade BoomHurry up!\n \nIndonesia EarnCryp...
298,1110293,Hey elon tomorrow world dog day pump!!!! dogec...


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,date,tweet,is_retweet,real_score,vader_pseudo_label,text,modded_text
0,1617175,1623460,NaN,2021-11-05 18:06:59,_73 Thank you for the opportunity to participa...,False,NaN,Positive,_73 Thank you for the opportunity to participa...,_73 Thank you for the opportunity to participa...
1,722687,725396,NaN,2021-08-09 21:26:20,"5 Top Crypto to Buy this Week [BTC, ETH, ADA, ...",False,NaN,Positive,"5 Top Crypto to Buy this Week [BTC, ETH, ADA, ...","5 Top Crypto to Buy this Week [BTC, ETH, ADA, ..."
2,1142504,1147171,NaN,2021-08-24 19:09:38,Huobi The World's 2nd Largest Cryptocurrency ...,False,NaN,Positive,Huobi The World's 2nd Largest Cryptocurrency ...,Huobi The World's 2nd Largest Cryptocurrency ...
3,1767599,1774085,NaN,2021-11-19 11:28:48,99%+ of traders don't care about Ferraris and ...,False,NaN,Positive,99%+ of traders don't care about Ferraris and ...,99%+ of traders don't care about Ferraris and ...
4,164807,165047,NaN,2021-06-22 19:03:58,I love this project so much! I hope the projec...,False,NaN,Positive,I love this project so much! I hope the projec...,I love this project so much! I hope the projec...
...,...,...,...,...,...,...,...,...,...,...
295,1692263,1698651,NaN,2021-11-12 19:17:59,"Episode 11 is live and “Bibles, Bullets, Bitco...",False,NaN,Neutral,"Episode 11 is live and “Bibles, Bullets, Bitco...","Episode 11 is live and Bibles, Bullets, Bitcoi..."
296,3298451,3308075,NaN,2022-05-10 21:32:02,Bitcoin tumbles more than 50% below its all-ti...,False,NaN,Neutral,Bitcoin tumbles more than 50% below its all-ti...,Bitcoin tumbles more than 50% below its all-ti...
297,2286522,2293761,NaN,2022-02-18 06:43:25,💥FaviTrade Boom💥Hurry up!\n \nIndonesia EarnCr...,False,NaN,Neutral,💥FaviTrade Boom💥Hurry up!\n \nIndonesia EarnCr...,FaviTrade BoomHurry up!\n \nIndonesia EarnCryp...
298,1110293,1114797,NaN,2021-08-25 22:19:49,Hey elon tomorrow world dog day pump!!!!🚀🚀 dog...,False,NaN,Neutral,Hey elon tomorrow world dog day pump!!!!🚀🚀 dog...,Hey elon tomorrow world dog day pump!!!! dogec...
